In [2]:
import numpy as np
import pandas as pd

Train_Beneficiarydata=pd.read_csv("../data/Train_Beneficiarydata-1542865627584.csv")
Test_Beneficiarydata=pd.read_csv("../data/Test_Beneficiarydata-1542969243754.csv")

In [3]:
Train_Beneficiarydata.info()
Test_Beneficiarydata.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138556 entries, 0 to 138555
Data columns (total 25 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   BeneID                           138556 non-null  object
 1   DOB                              138556 non-null  object
 2   DOD                              1421 non-null    object
 3   Gender                           138556 non-null  int64 
 4   Race                             138556 non-null  int64 
 5   RenalDiseaseIndicator            138556 non-null  object
 6   State                            138556 non-null  int64 
 7   County                           138556 non-null  int64 
 8   NoOfMonths_PartACov              138556 non-null  int64 
 9   NoOfMonths_PartBCov              138556 non-null  int64 
 10  ChronicCond_Alzheimer            138556 non-null  int64 
 11  ChronicCond_Heartfailure         138556 non-null  int64 
 12  ChronicCond_Kidn

BeneID                                 0
DOB                                    0
DOD                                63394
Gender                                 0
Race                                   0
RenalDiseaseIndicator                  0
State                                  0
County                                 0
NoOfMonths_PartACov                    0
NoOfMonths_PartBCov                    0
ChronicCond_Alzheimer                  0
ChronicCond_Heartfailure               0
ChronicCond_KidneyDisease              0
ChronicCond_Cancer                     0
ChronicCond_ObstrPulmonary             0
ChronicCond_Depression                 0
ChronicCond_Diabetes                   0
ChronicCond_IschemicHeart              0
ChronicCond_Osteoporasis               0
ChronicCond_rheumatoidarthritis        0
ChronicCond_stroke                     0
IPAnnualReimbursementAmt               0
IPAnnualDeductibleAmt                  0
OPAnnualReimbursementAmt               0
OPAnnualDeductib

In [4]:
# Our data has alot of stuff that eneds to change:
# 1. Alot of columns contain 2 which means no, so we replace it with 0
# 2. RenalDiseaseIndicator contains Y and 0 so we replace Y with 1
# 3. DOD has alot of null values, so we need to create multiple features to replace it, Age and IsDeceased are the idea now

In [ ]:
Train_Beneficiarydata = Train_Beneficiarydata.replace({'Gender':2, 'Race':2, 'ChronicCond_Alzheimer':2, 'ChronicCond_Heartfailure':2, 'ChronicCond_KidneyDisease':2, 'ChronicCond_Cancer':2, 'ChronicCond_ObstrPulmonary':2, 'ChronicCond_Depression':2, 'ChronicCond_Diabetes':2, 'ChronicCond_IschemicHeart':2, 'ChronicCond_Osteoporasis':2, 'ChronicCond_rheumatoidarthritis':2, 'ChronicCond_stroke':2}, 0)
Train_Beneficiarydata = Train_Beneficiarydata.replace({'RenalDiseaseIndicator':'Y'}, 1)
Test_Beneficiarydata = Test_Beneficiarydata.replace({'Gender':2, 'Race':2, 'ChronicCond_Alzheimer':2, 'ChronicCond_Heartfailure':2, 'ChronicCond_KidneyDisease':2, 'ChronicCond_Cancer':2, 'ChronicCond_ObstrPulmonary':2, 'ChronicCond_Depression':2, 'ChronicCond_Diabetes':2, 'ChronicCond_IschemicHeart':2, 'ChronicCond_Osteoporasis':2, 'ChronicCond_rheumatoidarthritis':2, 'ChronicCond_stroke':2}, 0)
Test_Beneficiary = Test_Beneficiarydata.replace({'RenalDiseaseIndicator':'Y'}, 1)
Train_Beneficiarydata['DOB'] = pd.to_datetime(Train_Beneficiarydata['DOB'], errors='ignore',format='%Y-%m-%d')
Test_Beneficiarydata['DOB'] = pd.to_datetime(Test_Beneficiarydata['DOB'], errors='ignore',format='%Y-%m-%d')

Train_Beneficiarydata['DOD'] = pd.to_datetime(Test_Beneficiarydata['DOD'], errors='ignore',format='%Y-%m-%d')
Test_Beneficiarydata['DOD'] = pd.to_datetime(Test_Beneficiarydata['DOD'], errors='ignore',format='%Y-%m-%d')
# Create age and calculate for the dead
Train_Beneficiarydata['Age'] = (Test_Beneficiary['DOD']-Train_Beneficiarydata['DOB']).dt.days // 365
Test_Beneficiarydata['Age'] = (Test_Beneficiarydata['DOD']-Test_Beneficiarydata['DOB']).dt.days // 365

C:\Users\omarh\AppData\Local\Temp\ipykernel_28812\451885768.py:5: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  Train_Beneficiarydata['DOB'] = pd.to_datetime(Train_Beneficiarydata['DOB'], errors='ignore',format='%Y-%m-%d')
C:\Users\omarh\AppData\Local\Temp\ipykernel_28812\451885768.py:6: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  Test_Beneficiarydata['DOB'] = pd.to_datetime(Test_Beneficiarydata['DOB'], errors='ignore',format='%Y-%m-%d')
C:\Users\omarh\AppData\Local\Temp\ipykernel_28812\451885768.py:8: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  Train_Beneficiarydata['DOD'] = pd.to_datetime(Test_Beneficiarydata['DOD'], errors='ignore'

In [ ]:
# Fill alive patients with dataset's  (last death was 1/12/2009) - DOB 
Test_Beneficiarydata['Age']=Test_Beneficiarydata['Age'].fillna((pd.to_datetime('2009-12-01', format='%Y-%m-%d') - Test_Beneficiarydata['DOB']).dt.days // 365)

ValueError: time data "12/1/2009" doesn't match format "%m-%d-%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [8]:
Train_Beneficiarydata.head(10)

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age
0,BENE11001,1943-01-01,NaT,1,1,0,39,230,12,12,...,1,1,0,1,1,36000,3204,60,70,NaN
1,BENE11002,1936-09-01,2009-12-01,0,1,0,39,280,12,12,...,0,0,0,0,0,0,0,30,50,73.0
2,BENE11003,1936-08-01,NaT,1,1,0,52,590,12,12,...,0,1,0,0,0,0,0,90,40,NaN
3,BENE11004,1922-07-01,NaT,1,1,0,39,270,12,12,...,1,1,1,1,0,0,0,1810,760,NaN
4,BENE11005,1935-09-01,NaT,1,1,0,24,680,12,12,...,1,0,0,0,0,0,0,1790,1200,NaN
5,BENE11006,1976-09-01,NaT,0,1,0,23,810,12,12,...,0,0,0,0,0,0,0,500,0,NaN
6,BENE11007,1940-09-01,NaT,1,0,0,45,610,12,12,...,1,0,1,1,0,0,0,1490,160,NaN
7,BENE11008,1934-02-01,NaT,0,1,0,15,140,12,12,...,1,0,0,0,0,0,0,30,0,NaN
8,BENE11009,1929-06-01,NaT,1,1,1,44,230,12,12,...,1,0,0,0,0,0,0,100,0,NaN
9,BENE11010,1936-07-01,NaT,0,1,0,41,30,12,12,...,1,1,1,0,0,0,0,1170,660,NaN
